Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model, ablation_barplot, compare_fluxes

# Construct models of cells of interest, optimise

In [ ]:
BY4741 = Yeast8Model("./models/ecYeastGEM_batch.xml")
BY4741.make_auxotroph("BY4741")
BY4741.optimize()

zwf1 = Yeast8Model("./models/ecYeastGEM_batch.xml")
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.optimize()

# Ablate

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
ablation_barplot(BY4741.ablation_result, ax)
plt.show()

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
ablation_barplot(zwf1.ablation_result, ax)
plt.show()

In [ ]:
BY4741.ablation_result

In [ ]:
zwf1.ablation_result

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {BY4741.model.reactions.get_by_id(rxn_id).name}, {flux}')